# AuxTel Calibration Illumination System Functional Test

This notebook is meant to be used a functional checkout of the AuxTel illumination system. It includes all functionality but doesn't necessarily show how the system should be used in operation. It was written within the context of the Tucson Teststand and should be modified to be run on the summit.

In [ ]:
import asyncio

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.time import Time
from astropy.table import Table

from lsst_efd_client import EfdClient
client = EfdClient('summit_efd')

from lsst.ts import salobj

In [ ]:
# change this to a local directory
data_dir = '/home/parfa30/DATA'

In [ ]:
d = salobj.Domain()

### Connect to CSCs

In [ ]:
atmonochromator = salobj.Remote(d, 'ATMonochromator')
FiberSpectrograph = salobj.Remote(name="FiberSpectrograph", domain=d, index=3)
electrometer = salobj.Remote(name="Electrometer", domain=d, index=201) 
WhiteLightSource = salobj.Remote(name='ATWhiteLight', domain=d)

In [ ]:
await atmonochromator.start_task
await FiberSpectrograph.start_task
await electrometer.start_task
await WhiteLightSource.start_task

In [ ]:
async def get_status():
    mono_tmp = await atmonochromator.evt_summaryState.aget()
    spec_tmp = await FiberSpectrograph.evt_summaryState.aget()
    elec_tmp = await electrometer.evt_summaryState.aget()
    wls_tmp = await WhiteLightSource.evt_summaryState.aget()
    print('Monochromator: ',salobj.State(mono_tmp.summaryState))
    print('Fiber Spectrometer: ',salobj.State(spec_tmp.summaryState))
    print('Electrometer: ',salobj.State(elec_tmp.summaryState))
    print('WhiteLightSource: ',salobj.State(wls_tmp.summaryState))

In [ ]:
await get_status()

### Test White Light Source

Note: don't cycle the WLS between Standby and Enabled. You will have to wait 15 minutes on either side of turning it on and off. You can power cycle the other CSCs

In [ ]:
state = salobj.State.ENABLED
tmp = await salobj.set_summary_state(WhiteLightSource, state)

In [ ]:
#Start chiller
tmp = await WhiteLightSource.cmd_setChillerTemperature.set_start(temperature=20)
tmp = await WhiteLightSource.cmd_startChiller.set_start()
# Confirm that the chiller has started and running at 20C

In [ ]:
#Test shutters
tmp = await WhiteLightSource.cmd_closeShutter.set_start()

In [ ]:
tmp = await WhiteLightSource.cmd_openShutter.set_start()
#Keep shutter open so you can see the lamp turned on

In [ ]:
#Turn on lamp. It will then go into a warm up period before it will turn on 
tmp = await WhiteLightSource.cmd_turnLampOn.set_start(power = 800)

In [ ]:
LampBasicState = {0:'Unknown',1:'Off',2:'On',3:'?',4:'Cooldown',5:'Warmup'}
LampControllerError = {-1:'NoError',0:'Unknown',1:'KillSwitch',2:'ChassisOverheating',
                       3:'AccessDoor',4:'BallastOverheasting',5:'USBDisconnected',6:'AirflowMalfunction',
                       7:'LampStuckOn',8:'AirflowMalfunction'}
LampControllerState = {0:'Unknown',1:'Off',2:'StandbyOrOn',3:'Cooldown',4:'Error'}

In [ ]:
state = await WhiteLightSource.evt_lampState.aget()
print('Lamp State: ', LampBasicState[state.basicState])
print('Lamp Controller State: ', LampControllerState[state.controllerState])
print('Lamp Controller Error: ', LampControllerError[state.controllerError])

if LampBasicState[state.basicState] == 'Cooldown':
    time_left = state.cooldownEndTime - state.private_rcvStamp
    print('Time Left: {} min.'.format(time_left/60.))

elif LampBasicState[state.basicState] == 'Warmup':
    time_left = state.warmupEndTime - state.private_rcvStamp
    print('Time Left: {} min.'.format(time_left/60.))

In [ ]:
# When it is done warming up, turn up the lamp
tmp = await WhiteLightSource.cmd_turnLampOn.set_start(power=1000)

In [ ]:
# When you are done with all testing, turn off the White Light Source, 
tmp = await WhiteLightSource.cmd_turnLampOff.set_start()
## note, you can also send it a Force=True if there is some issue and you need to turn off the lamp

In [ ]:
# Wait 10 minutes, then turn off the Chiller
tmp = await await WhiteLightSource.cmd_stopChiller.set_start()

In [ ]:
# When everything is turned off, you can turn off power to the KiloArc and Chiller using the PDU
# auxtel-illpdu.cp.lsst.org

### Test Monochromator

In [ ]:
state = salobj.State.ENABLED
tmp = await salobj.set_summary_state(atmonochromator, state)

In [ ]:
async def get_params():
    tmp1 = await atmonochromator.evt_wavelength.aget()
    tmp2 = await atmonochromator.evt_entrySlitWidth.aget()
    tmp3 = await atmonochromator.evt_exitSlitWidth.aget()
    tmp4 = await atmonochromator.evt_selectedGrating.aget()
    return (tmp1.wavelength, tmp2.width, tmp3.width, tmp4.gratingType)

In [ ]:
await get_params()

In [ ]:
# Change wavelength. Confirm that the color changes
for wave in [400, 500, 600, 700]:
    await atmonochromator.cmd_changeWavelength.set_start(wavelength=wave)

In [ ]:
# Change slit size. Confirm that the brightness changes by eye
for entry in [0.5, 4.5]:
    for exit in [0.5, 4.5]:
        await atmonochromator.cmd_changeSlitWidth.set_start(slit=1, slitWidth=entry_width)
        await atmonochromator.cmd_changeSlitWidth.set_start(slit=2, slitWidth=exit_width)

In [ ]:
# Change grating. If this times out, send monochromator to STANDBY and then back to ENABLED. 
# It takes a long time to change the grating.
for grating in [0,1]:
    await atmonochromator.cmd_selectGrating.set_start(gratingType=grating, timeout=60)

### Test Electrometer

In [ ]:
state = salobj.State.ENABLED
tmp = await salobj.set_summary_state(electrometer, state, override='tts_cimacs3_v3.yaml',timeout=20)

In [ ]:
await electrometer.cmd_performZeroCalib.set_start(timeout=10)
await electrometer.cmd_setDigitalFilter.set_start(activateFilter=False, activateAvgFilter=False, activateMedFilter=False, timeout=10)    

In [ ]:
mode = await electrometer.evt_measureType.aget()
intTime = await electrometer.evt_integrationTime.aget()
range_ = await electrometer.evt_measureRange.aget()
print(f'Mode: {mode.mode}; Int. Time: {intTime.intTime}; Range: {range_.rangeValue}')

In [ ]:
exp_time = 1 #sec
etmp1=await electrometer.cmd_startScan.set_start(timeout=10)
await asyncio.sleep(exp_time)
tmp2=await electrometer.cmd_stopScan.set_start(timeout=10)
lfa = await electrometer.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
filename= os.path.split(lfa.url)[1]

In [ ]:
#transfer file to your machine so you can look at it
elec_filen = os.path.join(data_dir, filename)
os.system(f"curl {lfa.url} --output {elec_filen}")

In [ ]:
# Look at Data
hdu = fits.open(f'{elec_filen}')
data = hdu[1].data
print(data)

In [ ]:
# Plot Data
plt.plot(data['ElapsedTime'], data['Signal'])
plt.xlabel('ElapsedTime')
plt.ylabel('Signal')

### Test Fiber Spectrograph

In [ ]:
state = salobj.State.ENABLED
tmp = await salobj.set_summary_state(FiberSpectrograph, state)

In [ ]:
exp_time = 1 #sec
FiberSpectrograph.evt_largeFileObjectAvailable.flush()
tmp1 = await FiberSpectrograph.cmd_expose.set_start(duration=exp_time, numExposures=1)
lfa = await FiberSpectrograph.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
filename = os.path.split(lfa.url)[1]

In [ ]:
#transfer file to your machine so you can look at it
spec_filen = os.path.join(data_dir, filename)
os.system(f"curl {lfa.url} --output {spec_filen}")

In [ ]:
# Look at Data
hdu = fits.open(f'{elec_filen}')
wavelength = hdu[1].data['wavelength'].flatten()
spectra = hdu[0].data

In [ ]:
# Plot data
for spec in spectra: #there is probably only one
    plt.plot(wavelength, spec)
plt.xlabel('Wavelength (nm)')
plt.ylabel('Counts')

## Test all Components Together

In [ ]:
async def set_params(wave, entry_width, exit_width, grating):
    await atmonochromator.cmd_changeSlitWidth.set_start(slit=1, slitWidth=entry_width)
    await atmonochromator.cmd_changeSlitWidth.set_start(slit=2, slitWidth=exit_width)
    await atmonochromator.cmd_changeWavelength.set_start(wavelength=wave)
    await atmonochromator.cmd_selectGrating.set_start(gratingType=grating, timeout=60)
   
    tmp1 = await atmonochromator.evt_wavelength.aget()
    tmp2 = await atmonochromator.evt_entrySlitWidth.aget()
    tmp3 = await atmonochromator.evt_exitSlitWidth.aget()
    tmp4 = await atmonochromator.evt_selectedGrating.aget()
    return (tmp1.wavelength, tmp2.width, tmp3.width, tmp4.gratingType)

In [ ]:
def get_file_transfer(lfa):
    filename = os.path.split(lfa.url)[1]
    save_dir = os.path.join(data_dir, filename.split(':')[0])
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_file = os.path.join(save_dir, filename)
    os.system(f"curl {lfa.url} --output {save_file}")
    return save_file

In [ ]:
async def elec_meas(exp_time):
    tmp1=await electrometer.cmd_startScan.set_start(timeout=10)
    await asyncio.sleep(exp_time)
    tmp2=await electrometer.cmd_stopScan.set_start(timeout=10)
    lfa = await electrometer.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
    filename = get_file_transfer(lfa)
    return filename

In [ ]:
async def spect_meas(exp_time):
    tmp1 = await FiberSpectrograph.cmd_expose.set_start(duration=exp_time, numExposures=1)
    lfa = await FiberSpectrograph.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
    filename = get_file_transfer(lfa)
    return filename

In [ ]:
data = []
exp_time = 1.
start = Time.now()
for wls_power in [800, 900, 1000, 1100]:
    tmp = await WhiteLightSource.cmd_turnLampOn.set_start(power=wls_power)
    for wave in np.linspace(300, 1200, 10):
        print(wave, Time.now())
        for entry_width in [0.5, 4.9]: #[, 0.1]: #, 3., 4., 4.9]:
            for exit_width in [0.5, 4.9]: #[0.02,0.04, 0.06, 0.08, 0.1]: #, 3., 4., 4.9]:
                start_meas = Time.now().isot
                wavelength, entry_width, exit_width, gratingType = await set_params(wave, entry_width, exit_width, grating)
                await elec_meas(exp_time)   
                spec_fn = await spect_meas(exp_time)
                await asyncio.sleep(5.)
                end_meas = Time.now().isot
                data.append([start_meas, end_meas, wls_power, exp_time, wavelength, entry_width, exit_width, gratingType, spec_fn])

In [ ]:
data = []
exp_time = 1.
start = Time.now()
for wls_power in [800, 900, 1000, 1100]:
    tmp = await WhiteLightSource.cmd_turnLampOn.set_start(power=wls_power)
    for wave in np.linspace(500, 800, 3):
        print(wave, Time.now())
        for entry_width in [0.5, 4.9]: #[, 0.1]: #, 3., 4., 4.9]:
            for exit_width in [0.5, 4.9]: #[0.02,0.04, 0.06, 0.08, 0.1]: #, 3., 4., 4.9]:
                start_meas = Time.now().isot
                wavelength, entry_width, exit_width, gratingType = await set_params(wave, entry_width, exit_width, grating)
                elec_fn = await elec_meas(exp_time)   
                spec_fn = await spect_meas(exp_time)
                await asyncio.sleep(5.)
                end_meas = Time.now().isot
                data.append([start_meas, end_meas, wls_power, exp_time, wavelength, entry_width, exit_width, gratingType, spec_fn, elec_fn])

In [ ]:
df = pd.DataFrame(data, columns = ['start', 'end', 'wls_power', 'exp_time', 'wavelength','entry_slit', 'exit_slit', 
                                  'grating', 'spec_fn', 'elec_fn'])
t = Table.from_pandas(df)

### Save Data into a single fits file

In [ ]:
# Save data into a single fits file
elec_time = []
elec_data = []
wave = []
spectra = []

for row in t:
    spec_fn = row[-2]
    hdu = fits.open(spec_fn)
    spectra.append(hdu[0].data)
    wave.append(hdu[1].data['wavelength'].flatten())
    
    elec_fn = row[-1]
    hdu = fits.open(elec_fn)
    d = hdu[1].data
    elec_time.append(d['Elapsed Time'])
    elec_data.append(d['Signal'])
    
#Reshape the data to put it all in a fits file. Sorry, this is ugly
shape_elec = max([len(l) for l in elec_time])
shape_wave = max([len(l) for l in wave])

elec_time2 = np.zeros((len(elec_time), shape_elec))
elec_data2 = np.zeros((len(elec_time), shape_elec))
for i, row in enumerate(elec_time):
    elec_time2[i][0:row.shape[0]] = row
    elec_data2[i][0:row.shape[0]] = np.array(elec_data)[i]

hdulist = fits.HDUList()
empty_primary = fits.PrimaryHDU()
hdulist.append(empty_primary)

#Make a rec array of the data
hdulist.append(fits.BinTableHDU(t.as_array(), name='CONFIG'))

hdulist.append(fits.ImageHDU(np.vstack(elec_time2), name='ELEC_TIME'))
hdulist.append(fits.ImageHDU(np.vstack(elec_data2), name='ELEC_DATA'))
hdulist.append(fits.ImageHDU(np.vstack(wave), name='WAVELENGTH'))
hdulist.append(fits.ImageHDU(np.vstack(spectra), name='SPECTRA'))

save_filen = '{}/{}.fits'.format(os.path.join(data_dir,'AuxTel'), 'test_20221209')
hdulist.writeto(save_filen, overwrite=True)
print('Data Saved to {}'.format(save_filen))

In [ ]:
hdu = fits.open(save_filen)
hdu.info()

In [ ]:
Table(hdu[1].data)

In [ ]:
waves = hdu[4].data
spectra = hdu[5].data
ret = plt.plot(waves, spectra)